<a href="https://colab.research.google.com/github/zelal-Eizaldeen/DLH_Project_HurtfulWords_Paper/blob/main/4_GroupSents_pc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Import modules
import sys
sys.path.append('/content/drive/MyDrive/Payel-DLH-related/Colab Notebooks')

Download heuristic_tokenize.py from HurtfulWords/scripts and upload to Colab Notebooks folder

In [ ]:
import pandas as pd
import numpy as np
# from pytorch_pretrained_bert.tokenization import BertTokenizer
from transformers import BertTokenizer
import random
# import argparse

This is what comea AS INPUT TO THIS SCRIPT:

In [ ]:
# Column Name	    Description
# text	          Original text
# sents	          Tokenized sentences
# sections	      Cleaned full text sections
# mod_text	      Processed full text
# toks	          Tokenized document (BERT)
# num_toks	      Number of tokens in toks
# sent_toks_lens	List of token counts per sentence

Found existing installation: spacy 3.7.5
Uninstalling spacy-3.7.5:
  Successfully uninstalled spacy-3.7.5
  Using cached spacy-2.3.5.tar.gz (5.8 MB)
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
  Using cached spacy-3.7.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (27 kB)
Using cached spacy-3.7.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.6 MB)


In [ ]:
!pwd

/content


INstall Scispacy and download en_core_sci_md  #Downloaded from https://allenai.github.io/scispacy/ and then upload


In [ ]:
class Args:
    input_loc = "/content/drive/MyDrive/Payel-DLH-related/DataFiles/sampled_files/df_extract.pkl"  # Update with actual file path
    output_loc = "/content/drive/MyDrive/Payel-DLH-related/DataFiles/sampled_files/df_grouped.pkl"
    model_path = '/content/drive/MyDrive/Payel-DLH-related/HurtfulWords/pretrainedModels/Baseline_Clinical_BERT/baseline_clinical_BERT_1_epoch_512'
    under_prob = 0  # Default as in original script
    minlen = [20]  # Default as in original script

args = Args()  # Mimic argparse behavior

In [ ]:
baseline_tokenizer = BertTokenizer.from_pretrained(args.model_path)
# print("Lowercasing Enabled:", baseline_tokenizer.basic_tokenizer.do_lower_case)

Lowercasing Enabled: True


In [ ]:
df = pd.read_pickle(args.input_loc)

In [ ]:
df.head()

,row_id,subject_id_x,hadm_id,chartdate,charttime,storetime,category,description,cgid,iserror,...,age,icd9_code,language_to_use,icustay_id,sents,sections,mod_text,toks,num_toks,sent_toks_lens
0,37,32779,133757.0,2167-10-27,NaN,NaN,Discharge summary,Report,NaN,NaN,...,0.0,"[51881, 41071, 5849, 2762, 4280, 4275, 486, 49...",Other,None,"[Admission Date: PHIDATEPHI, Discharge Date: P...","[(Admission, Date, :, PHIDATEPHI), (Discharge,...",Admission Date: PHIDATEPHI\nDischarge Date: PH...,"[admission, date, :, phidatephi, discharge, da...",3468,"[4, 4, 5, 3, 3, 3, 22, 3, 7, 9, 5, 29, 1, 22, ..."
1,444,15159,187095.0,2103-08-19,NaN,NaN,Discharge summary,Report,NaN,NaN,...,0.0,"[486, 2765, 29900, 7580]",Missing,None,"[Admission Date: PHIDATEPHI, Discharge Date: P...","[(Admission, Date, :, PHIDATEPHI), (Discharge,...",Admission Date: PHIDATEPHI\nDischarge Date: PH...,"[admission, date, :, phidatephi, discharge, da...",1060,"[4, 4, 5, 3, 2, 10, 5, 48, 22, 7, 16, 4, 10, 2..."
2,753,82122,100504.0,2175-02-04,NaN,NaN,Discharge summary,Report,NaN,NaN,...,0.0,"[41091, 42823, 5849, 4271, 2761, 2762, V667, 4...",Other,None,"[Admission Date: PHIDATEPHI, Discharge Date: P...","[(Admission, Date, :, PHIDATEPHI), (Discharge,...",Admission Date: PHIDATEPHI\nDischarge Date: PH...,"[admission, date, :, phidatephi, discharge, da...",3915,"[4, 4, 5, 3, 3, 11, 3, 10, 7, 5, 36, 14, 16, 2..."
3,494,16447,163590.0,2191-09-03,NaN,NaN,Discharge summary,Report,NaN,NaN,...,0.0,"[4275, 4822, 3481, 2967, V182]",Missing,None,"[Admission Date: PHIDATEPHI, Discharge Date: P...","[(Admission, Date, :, PHIDATEPHI), (Discharge,...",Admission Date: PHIDATEPHI\nDischarge Date: PH...,"[admission, date, :, phidatephi, discharge, da...",911,"[4, 4, 5, 3, 8, 11, 5, 34, 9, 15, 9, 12, 9, 16..."
4,908,31778,186882.0,2101-07-29,NaN,NaN,Discharge summary,Report,NaN,NaN,...,0.0,"[0383, 51881, 42831, 00845, 78552, 5119, 5849,...",English,None,"[Admission Date: PHIDATEPHI, Discharge Date: P...","[(Admission, Date, :, PHIDATEPHI), (Discharge,...",Admission Date: PHIDATEPHI\nDischarge Date: PH...,"[admission, date, :, phidatephi, discharge, da...",1961,"[4, 4, 5, 3, 3, 13, 3, 4, 6, 22, 5, 50, 40, 9,..."


In [ ]:
def pack_sentences(row, minlen):
    i, cumsum, init = 0,0,0
    seqs, tok_len_sums = [], []
    while i<len(row.sent_toks_lens):
        cumsum += row.sent_toks_lens[i]
        if cumsum>= minlen:
            if init == i or random.random() >= args.under_prob:
                seqs.append('\n'.join(row.sents[init:i+1]))
            else: #roll back one
                seqs.append('\n'.join(row.sents[init:i]))
                cumsum -= row.sent_toks_lens[i]
                i -=1
            tok_len_sums.append(cumsum)
            cumsum = 0
            init = i+1
        i+=1
    if init != i:
        seqs.append('\n'.join(row.sents[init:]))
        tok_len_sums.append(cumsum)
    return [seqs, tok_len_sums]

In [ ]:
for i in args.minlen:
	df['BERT_sents'+str(i)], df['BERT_sents_lens'+str(i)] = zip(*df.apply(pack_sentences, axis = 1, minlen = i))
	df['num_BERT_sents'+str(i)] = df['BERT_sents'+str(i)].apply(len)
	assert(all(df['BERT_sents_lens'+str(i)].apply(sum) == df['sent_toks_lens'].apply(sum)))

In [ ]:
df.to_pickle(args.output_loc)

In [ ]:
df.head()

,row_id,subject_id_x,hadm_id,chartdate,charttime,storetime,category,description,cgid,iserror,...,icustay_id,sents,sections,mod_text,toks,num_toks,sent_toks_lens,BERT_sents20,BERT_sents_lens20,num_BERT_sents20
0,37,32779,133757.0,2167-10-27,NaN,NaN,Discharge summary,Report,NaN,NaN,...,None,"[Admission Date: PHIDATEPHI, Discharge Date: P...","[(Admission, Date, :, PHIDATEPHI), (Discharge,...",Admission Date: PHIDATEPHI\nDischarge Date: PH...,"[admission, date, :, phidatephi, discharge, da...",3468,"[4, 4, 5, 3, 3, 3, 22, 3, 7, 9, 5, 29, 1, 22, ...",[Admission Date: PHIDATEPHI\nDischarge Date: P...,"[22, 22, 24, 29, 23, 22, 38, 28, 38, 32, 26, 3...",99
1,444,15159,187095.0,2103-08-19,NaN,NaN,Discharge summary,Report,NaN,NaN,...,None,"[Admission Date: PHIDATEPHI, Discharge Date: P...","[(Admission, Date, :, PHIDATEPHI), (Discharge,...",Admission Date: PHIDATEPHI\nDischarge Date: PH...,"[admission, date, :, phidatephi, discharge, da...",1060,"[4, 4, 5, 3, 2, 10, 5, 48, 22, 7, 16, 4, 10, 2...",[Admission Date: PHIDATEPHI\nDischarge Date: P...,"[28, 53, 22, 23, 41, 38, 42, 22, 33, 27, 24, 2...",33
2,753,82122,100504.0,2175-02-04,NaN,NaN,Discharge summary,Report,NaN,NaN,...,None,"[Admission Date: PHIDATEPHI, Discharge Date: P...","[(Admission, Date, :, PHIDATEPHI), (Discharge,...",Admission Date: PHIDATEPHI\nDischarge Date: PH...,"[admission, date, :, phidatephi, discharge, da...",3915,"[4, 4, 5, 3, 3, 11, 3, 10, 7, 5, 36, 14, 16, 2...",[Admission Date: PHIDATEPHI\nDischarge Date: P...,"[30, 20, 41, 30, 25, 25, 24, 42, 38, 46, 35, 2...",128
3,494,16447,163590.0,2191-09-03,NaN,NaN,Discharge summary,Report,NaN,NaN,...,None,"[Admission Date: PHIDATEPHI, Discharge Date: P...","[(Admission, Date, :, PHIDATEPHI), (Discharge,...",Admission Date: PHIDATEPHI\nDischarge Date: PH...,"[admission, date, :, phidatephi, discharge, da...",911,"[4, 4, 5, 3, 8, 11, 5, 34, 9, 15, 9, 12, 9, 16...",[Admission Date: PHIDATEPHI\nDischarge Date: P...,"[24, 50, 24, 21, 25, 21, 35, 21, 30, 22, 21, 2...",31
4,908,31778,186882.0,2101-07-29,NaN,NaN,Discharge summary,Report,NaN,NaN,...,None,"[Admission Date: PHIDATEPHI, Discharge Date: P...","[(Admission, Date, :, PHIDATEPHI), (Discharge,...",Admission Date: PHIDATEPHI\nDischarge Date: PH...,"[admission, date, :, phidatephi, discharge, da...",1961,"[4, 4, 5, 3, 3, 13, 3, 4, 6, 22, 5, 50, 40, 9,...",[Admission Date: PHIDATEPHI\nDischarge Date: P...,"[32, 35, 55, 40, 38, 31, 39, 31, 24, 47, 29, 2...",54


In [ ]:
%cd '/content/drive/MyDrive/Payel-DLH-related/DataFiles/sampled_files'
df.to_csv('df_grouped_view.csv', index=False)

/content/drive/MyDrive/Payel-DLH-related/DataFiles/sampled_files


BERT_sents20 contains the packed sentences.

BERT_sents_lens20 contains the token lengths of those packed sequences.

num_BERT_sents20 contains the number of packed sequences.


Let's assume you have a row where the original sentences are shorter than the minlen of 20 tokens:

Original sentences (sents):

["This is a short sentence.", "Here's another sentence.", "This one is a bit longer."]

Sentence token lengths (sent_toks_lens):

[5, 4, 6] (where each number represents the token count of the corresponding sentence in sents).

After applying the packing with minlen=20, you may get:

Packed sentences (BERT_sents20):

["This is a short sentence. Here's another sentence. This one is a bit longer."] (all sentences packed together into one sequence, since the total token count is 15 and does not exceed minlen=20).

Token lengths of packed sentences (BERT_sents_lens20):

[15] (since the total number of tokens in the packed sequence is 15).

Number of packed sentences (num_BERT_sents20):

1 (since only one sequence was created).